In [ ]:
%pip install transformers torch sympy matplotlib opencv-python numpy diffusers torch torchvision
%pip install openai gym[pybullet] manim pydantic requests accelerate
%pip install pytorch-lightning scikit-learn datasets nltk sentencepiece

%pip install transformers torch torchvision torchaudio diffusers accelerate
%pip install opencv-python numpy matplotlib scikit-image rouge-score
%pip install torchsummary pytorch-fid
%pip install gym[pybullet] manim openai

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import json

class ProblemClassifier:
    def __init__(self, model_path="bert-base-uncased"):
        self.tokenizer = BertTokenizer.from_pretrained(model_path)
        self.model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)  # Easy, Medium, Hard
        self.label_map = {"Easy": 0, "Medium": 1, "Hard": 2}  # Mapping difficulty to labels

    def classify_problem(self, text):
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = self.model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=1).item()
        return list(self.label_map.keys())[prediction]

# Load the dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Preprocess dataset for classification
classifier = ProblemClassifier()
for problem in dataset:
    input_text = problem["Problem_Statement"] + " Concepts: " + problem["Concepts_Covered"]
    predicted_difficulty = classifier.classify_problem(input_text)
    print(f"Problem ID: {problem['Problem_ID']}, Predicted Difficulty: {predicted_difficulty}, Actual: {problem['Difficulty_Level']}")


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import json

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Map difficulty levels to numerical labels
label_map = {"Easy": 0, "Medium": 1, "Hard": 2}

def preprocess_data(data):
    processed_data = []
    for problem in data:
        input_text = problem["Problem_Statement"] + " Concepts: " + problem["Concepts_Covered"]
        label = label_map[problem["Difficulty_Level"]]
        processed_data.append({"text": input_text, "label": label})
    return processed_data

# Preprocess dataset
data = preprocess_data(dataset)
dataset = Dataset.from_list(data)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

# Training arguments
training_args = TrainingArguments(
    output_dir="./bert_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

# Save fine-tuned model
model.save_pretrained("./bert_finetuned")
tokenizer.save_pretrained("./bert_finetuned")

print("Fine-tuning complete. Model saved to ./bert_finetuned")


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def load_finetuned_model(model_path="./bert_finetuned"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = problem_statement + " Concepts: " + concepts
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

# Load the fine-tuned model
tokenizer, model = load_finetuned_model()

# Example problem to test
example_problem = "A ball is thrown vertically upward with an initial velocity of 20 meters per second. Calculate the maximum height reached."
example_concepts = "Kinematics, free fall, acceleration due to gravity."

# Classify difficulty
predicted_difficulty = classify_problem(example_problem, example_concepts, tokenizer, model)
print(f"Predicted Difficulty: {predicted_difficulty}")


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import json

# Load the fine-tuned model
def load_finetuned_model(model_path="./bert_finetuned"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = problem_statement + " Concepts: " + concepts
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction

# Load dataset for evaluation
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

label_map = {"Easy": 0, "Medium": 1, "Hard": 2}
reverse_label_map = {0: "Easy", 1: "Medium", 2: "Hard"}

tokenizer, model = load_finetuned_model()

# Evaluate model
correct = 0
total = len(dataset)

for problem in dataset:
    actual_label = label_map[problem["Difficulty_Level"]]
    predicted_label = classify_problem(problem["Problem_Statement"], problem["Concepts_Covered"], tokenizer, model)
    if actual_label == predicted_label:
        correct += 1

accuracy = (correct / total) * 100
print(f"Model Accuracy: {accuracy:.2f}%")


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import json

# Load the fine-tuned model
def load_finetuned_model(model_path="./bert_finetuned"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = problem_statement + " Concepts: " + concepts
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction

# Load dataset for evaluation
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

label_map = {"Easy": 0, "Medium": 1, "Hard": 2}
reverse_label_map = {0: "Easy", 1: "Medium", 2: "Hard"}

tokenizer, model = load_finetuned_model()

# Evaluate model
correct = 0
total = len(dataset)

for problem in dataset:
    actual_label = label_map[problem["Difficulty_Level"]]
    predicted_label = classify_problem(problem["Problem_Statement"], problem["Concepts_Covered"], tokenizer, model)
    if actual_label == predicted_label:
        correct += 1

accuracy = (correct / total) * 100
print(f"Model Accuracy: {accuracy:.2f}%")


In [ ]:
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from transformers import BertTokenizer, BertForSequenceClassification
import json
import numpy as np

# Load the fine-tuned model
def load_finetuned_model(model_path="./bert_finetuned"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = problem_statement + " Concepts: " + concepts
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction

# Load dataset for evaluation
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

label_map = {"Easy": 0, "Medium": 1, "Hard": 2}
reverse_label_map = {0: "Easy", 1: "Medium", 2: "Hard"}

tokenizer, model = load_finetuned_model()

# Collect true and predicted labels
y_true = []
y_pred = []

for problem in dataset:
    actual_label = label_map[problem["Difficulty_Level"]]
    predicted_label = classify_problem(problem["Problem_Statement"], problem["Concepts_Covered"], tokenizer, model)
    y_true.append(actual_label)
    y_pred.append(predicted_label)

# Compute confusion matrix
cm = confusion_matrix(y_true, y_pred)
labels = ["Easy", "Medium", "Hard"]

def plot_confusion_matrix(cm, labels):
    plt.figure(figsize=(6,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()

# Display results
plot_confusion_matrix(cm, labels)
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=labels))


In [ ]:
import torch
import json
from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd

# Load the fine-tuned model
def load_finetuned_model(model_path="./bert_finetuned"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = problem_statement + " Concepts: " + concepts
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction

# Load dataset for evaluation
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

label_map = {"Easy": 0, "Medium": 1, "Hard": 2}
reverse_label_map = {0: "Easy", 1: "Medium", 2: "Hard"}

tokenizer, model = load_finetuned_model()

# Collect true and predicted labels
misclassified = []

for problem in dataset:
    actual_label = label_map[problem["Difficulty_Level"]]
    predicted_label = classify_problem(problem["Problem_Statement"], problem["Concepts_Covered"], tokenizer, model)
    if actual_label != predicted_label:
        misclassified.append({
            "Problem_ID": problem["Problem_ID"],
            "Problem_Statement": problem["Problem_Statement"],
            "Concepts_Covered": problem["Concepts_Covered"],
            "Actual Difficulty": reverse_label_map[actual_label],
            "Predicted Difficulty": reverse_label_map[predicted_label]
        })

# Convert misclassifications to a DataFrame and display
misclassified_df = pd.DataFrame(misclassified)
print("Misclassified Problems:")
print(misclassified_df)
misclassified_df.to_csv("misclassified_problems.csv", index=False)
print("Misclassification analysis saved to misclassified_problems.csv")


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
import pandas as pd

# Load FLAN-T5 model and tokenizer
def load_flan_t5(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def generate_solution(problem_statement, concepts, tokenizer, model, max_length=200):
    input_text = f"Solve step-by-step: {problem_statement} Concepts: {concepts}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

tokenizer, model = load_flan_t5()

# Generate solutions for each problem
solutions = []

for problem in dataset:
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    concepts = problem["Concepts_Covered"]
    generated_solution = generate_solution(problem_statement, concepts, tokenizer, model)
    solutions.append({
        "Problem_ID": problem_id,
        "Problem_Statement": problem_statement,
        "Generated_Solution": generated_solution,
        "Final_Answer": problem.get("Final_Answer", "N/A")
    })

# Save results
solutions_df = pd.DataFrame(solutions)
solutions_df.to_csv("flan_t5_generated_solutions.csv", index=False)
print("Generated solutions saved to flan_t5_generated_solutions.csv")


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from rouge_score import rouge_scorer

# Load generated solutions and actual answers
df = pd.read_csv("flan_t5_generated_solutions.csv")

def compute_rouge(reference, prediction):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, prediction)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

# Evaluate generated solutions
rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
for _, row in df.iterrows():
    reference = str(row["Final_Answer"])
    prediction = str(row["Generated_Solution"])
    r1, r2, rL = compute_rouge(reference, prediction)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rL)

df["ROUGE-1"] = rouge1_scores
df["ROUGE-2"] = rouge2_scores
df["ROUGE-L"] = rougeL_scores

df.to_csv("flan_t5_evaluation_results.csv", index=False)
print("Evaluation complete. Results saved to flan_t5_evaluation_results.csv")


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import json

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Prepare dataset for fine-tuning
def preprocess_data(data):
    processed_data = []
    for problem in data:
        input_text = f"Solve step-by-step: {problem['Problem_Statement']} Concepts: {problem['Concepts_Covered']}"
        target_text = problem.get("Final_Answer", "")
        processed_data.append({"input_text": input_text, "target_text": target_text})
    return processed_data

data = preprocess_data(dataset)
dataset = Dataset.from_list(data)

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")

def tokenize_function(examples):
    inputs = tokenizer(examples["input_text"], truncation=True, padding="max_length", max_length=512)
    targets = tokenizer(examples["target_text"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Load FLAN-T5 model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large")

# Training arguments
training_args = TrainingArguments(
    output_dir="./flan_t5_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

# Save fine-tuned model
model.save_pretrained("./flan_t5_finetuned")
tokenizer.save_pretrained("./flan_t5_finetuned")

print("Fine-tuning complete. Model saved to ./flan_t5_finetuned")


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load fine-tuned FLAN-T5 model
def load_finetuned_model(model_path="./flan_t5_finetuned"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def generate_solution(problem_statement, concepts, tokenizer, model, max_length=200):
    input_text = f"Solve step-by-step: {problem_statement} Concepts: {concepts}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Load the fine-tuned model
tokenizer, model = load_finetuned_model()

# Example test problem
example_problem = "A ball is thrown vertically upward with an initial velocity of 20 meters per second. Calculate the maximum height reached."
example_concepts = "Kinematics, free fall, acceleration due to gravity."

# Generate solution
generated_solution = generate_solution(example_problem, example_concepts, tokenizer, model)
print("Generated Solution:")
print(generated_solution)


In [ ]:
import torch
import json
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load fine-tuned FLAN-T5 model
def load_finetuned_model(model_path="./flan_t5_finetuned"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def generate_solution(problem_statement, concepts, tokenizer, model, max_length=200):
    input_text = f"Solve step-by-step: {problem_statement} Concepts: {concepts}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Load the fine-tuned model
tokenizer, model = load_finetuned_model()

# Load dataset for batch testing
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Generate solutions for all problems
solutions = []
for problem in dataset:
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    concepts = problem["Concepts_Covered"]
    generated_solution = generate_solution(problem_statement, concepts, tokenizer, model)
    solutions.append({
        "Problem_ID": problem_id,
        "Problem_Statement": problem_statement,
        "Generated_Solution": generated_solution,
        "Final_Answer": problem.get("Final_Answer", "N/A")
    })

# Save results to CSV
solutions_df = pd.DataFrame(solutions)
solutions_df.to_csv("flan_t5_batch_generated_solutions.csv", index=False)
print("Batch testing complete. Results saved to flan_t5_batch_generated_solutions.csv")


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from rouge_score import rouge_scorer

# Load batch test results
df = pd.read_csv("flan_t5_batch_generated_solutions.csv")

def compute_rouge(reference, prediction):
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, prediction)
    return scores["rouge1"].fmeasure, scores["rouge2"].fmeasure, scores["rougeL"].fmeasure

# Evaluate generated solutions
rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
for _, row in df.iterrows():
    reference = str(row["Final_Answer"])
    prediction = str(row["Generated_Solution"])
    r1, r2, rL = compute_rouge(reference, prediction)
    rouge1_scores.append(r1)
    rouge2_scores.append(r2)
    rougeL_scores.append(rL)

df["ROUGE-1"] = rouge1_scores
df["ROUGE-2"] = rouge2_scores
df["ROUGE-L"] = rougeL_scores

df.to_csv("flan_t5_batch_evaluation_results.csv", index=False)
print("Batch evaluation complete. Results saved to flan_t5_batch_evaluation_results.csv")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load evaluation results
df = pd.read_csv("flan_t5_batch_evaluation_results.csv")

# Plot ROUGE score distributions
def plot_rouge_scores(df):
    plt.figure(figsize=(12, 5))
    sns.histplot(df["ROUGE-1"], bins=20, kde=True, label="ROUGE-1", color='blue')
    sns.histplot(df["ROUGE-2"], bins=20, kde=True, label="ROUGE-2", color='green')
    sns.histplot(df["ROUGE-L"], bins=20, kde=True, label="ROUGE-L", color='red')
    plt.xlabel("ROUGE Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of ROUGE Scores")
    plt.legend()
    plt.show()

# Call the plotting function
plot_rouge_scores(df)

# Print summary statistics
print("Summary of ROUGE Scores:")
print(df[["ROUGE-1", "ROUGE-2", "ROUGE-L"]].describe())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load pre-finetuning and post-finetuning evaluation results
pre_finetune_df = pd.read_csv("flan_t5_batch_evaluation_results_pre_finetune.csv")
post_finetune_df = pd.read_csv("flan_t5_batch_evaluation_results.csv")

# Merge datasets for comparison
pre_finetune_df["Model"] = "Pre-Finetuned"
post_finetune_df["Model"] = "Post-Finetuned"
comparison_df = pd.concat([pre_finetune_df, post_finetune_df])

# Plot ROUGE score comparisons
def plot_rouge_comparison(df):
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="Model", y="ROUGE-1", data=df, palette=["blue", "green"])
    plt.title("ROUGE-1 Score Comparison Before and After Finetuning")
    plt.show()
    
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="Model", y="ROUGE-2", data=df, palette=["blue", "green"])
    plt.title("ROUGE-2 Score Comparison Before and After Finetuning")
    plt.show()
    
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="Model", y="ROUGE-L", data=df, palette=["blue", "green"])
    plt.title("ROUGE-L Score Comparison Before and After Finetuning")
    plt.show()

# Call the comparison function
plot_rouge_comparison(comparison_df)

# Print summary statistics comparison
print("Summary of ROUGE Scores Before and After Finetuning:")
print(comparison_df.groupby("Model")[["ROUGE-1", "ROUGE-2", "ROUGE-L"]].describe())


In [ ]:
import torch
import json
from transformers import LlamaTokenizer, LlamaForCausalLM

# Load LLaMA model and tokenizer
def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

# Preprocess the Physics Problem Dataset
def preprocess_physics_data(dataset_path="updated_physics_problems_dataset_2.json"):
    with open(dataset_path, "r", encoding="utf-8") as f:
        dataset = json.load(f)
    processed_data = []
    for problem in dataset:
        input_text = f"Physics Problem: {problem['Problem_Statement']}\nConcepts: {problem['Concepts_Covered']}"
        processed_data.append({"input_text": input_text, "answer": problem.get("Final_Answer", "N/A")})
    return processed_data

def generate_tutoring_response(question, context, tokenizer, model, max_length=300):
    input_text = f"Student Question: {question}\nContext: {context}\nAI Tutor Response:"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Load the LLaMA model
tokenizer, model = load_llama_model()

# Load and preprocess dataset
dataset = preprocess_physics_data()

# Example student interaction
example_question = "Can you explain Newton's laws of motion in a simple way?"
example_context = "Newton's laws describe the relationship between the motion of an object and the forces acting on it."

# Generate AI tutor response
tutor_response = generate_tutoring_response(example_question, example_context, tokenizer, model)
print("AI Tutor Response:")
print(tutor_response)

# Evaluate AI Tutoring Performance (Placeholder for ROUGE/BLEU Scoring)
def evaluate_response(reference, generated):
    from rouge_score import rouge_scorer
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

# Example evaluation
if dataset:
    reference_answer = dataset[0]["answer"]
    evaluation_scores = evaluate_response(reference_answer, tutor_response)
    print("Evaluation Scores:", evaluation_scores)


In [ ]:
import torch
import json
from diffusers import StableDiffusionPipeline
from transformers import BertTokenizer, BertForSequenceClassification
from torchvision import transforms
from PIL import Image

def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)  # Easy, Medium, Hard
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = f"{problem_statement} Concepts: {concepts}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_image(prompt, model_path="CompVis/stable-diffusion-v1-4"):
    pipe = StableDiffusionPipeline.from_pretrained(model_path)
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")
    image = pipe(prompt).images[0]
    return image

def apply_pix2pix_style(image, style_model_path="style_transfer_pix2pix.pth"):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    input_image = transform(image).unsqueeze(0)
    pix2pix_model = torch.jit.load(style_model_path)
    pix2pix_model.eval()
    with torch.no_grad():
        output_image = pix2pix_model(input_image).squeeze().detach().cpu()
    output_image = transforms.ToPILImage()(output_image)
    return output_image

# Load models
tokenizer, bert_model = load_bert_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Process dataset
for problem in dataset:
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    concepts = problem["Concepts_Covered"]
    difficulty = classify_problem(problem_statement, concepts, tokenizer, bert_model)
    image_prompt = problem["3D_Diagram_Description"]
    
    print(f"Generating image for Problem {problem_id} ({difficulty} difficulty)...")
    generated_image = generate_image(image_prompt)
    
    print("Applying style transfer...")
    styled_image = apply_pix2pix_style(generated_image)
    
    styled_image.save(f"generated_images/problem_{problem_id}_styled.png")
    print(f"Image saved: generated_images/problem_{problem_id}_styled.png")


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

def load_images_from_folder(folder_path, num_images=9):
    images = []
    filenames = sorted(os.listdir(folder_path))[:num_images]
    for filename in filenames:
        img_path = os.path.join(folder_path, filename)
        img = Image.open(img_path)
        images.append((filename, img))
    return images

def plot_image_grid(images, grid_size=(3, 3)):
    fig, axes = plt.subplots(grid_size[0], grid_size[1], figsize=(10, 10))
    fig.suptitle("Generated Physics Problem Illustrations", fontsize=16)
    for ax, (filename, img) in zip(axes.flat, images):
        ax.imshow(img)
        ax.set_title(filename)
        ax.axis("off")
    plt.show()

# Load and display images
image_folder = "generated_images"
images = load_images_from_folder(image_folder, num_images=9)
plot_image_grid(images)


In [ ]:
import torch
import json
import os
from diffusers import StableDiffusionPipeline
from transformers import BertTokenizer, BertForSequenceClassification
from torchvision import transforms
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

# Load BERT model for classification
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)  # Easy, Medium, Hard
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = f"{problem_statement} Concepts: {concepts}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_image(prompt, model_path="CompVis/stable-diffusion-v1-4"):
    pipe = StableDiffusionPipeline.from_pretrained(model_path)
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")
    image = pipe(prompt).images[0]
    return image

def save_image(image, path):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    image.save(path)

def process_problem(problem, tokenizer, bert_model, model_path="CompVis/stable-diffusion-v1-4"):
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    concepts = problem["Concepts_Covered"]
    difficulty = classify_problem(problem_statement, concepts, tokenizer, bert_model)
    image_prompt = problem["3D_Diagram_Description"]
    
    print(f"Generating image for Problem {problem_id} ({difficulty} difficulty)...")
    generated_image = generate_image(image_prompt, model_path)
    save_image(generated_image, f"generated_images/problem_{problem_id}.png")
    print(f"Image saved: generated_images/problem_{problem_id}.png")

# Load models
tokenizer, bert_model = load_bert_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Generate images in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    for problem in dataset:
        executor.submit(process_problem, problem, tokenizer, bert_model)


In [ ]:
import torch
import json
import os
import logging
from diffusers import StableDiffusionPipeline
from transformers import BertTokenizer, BertForSequenceClassification
from torchvision import transforms
from PIL import Image
from concurrent.futures import ThreadPoolExecutor

# Setup logging
logging.basicConfig(filename='image_generation.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

# Load BERT model for classification
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)  # Easy, Medium, Hard
    return tokenizer, model

def classify_problem(problem_statement, concepts, tokenizer, model):
    input_text = f"{problem_statement} Concepts: {concepts}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_image(prompt, model_path="CompVis/stable-diffusion-v1-4"):
    try:
        pipe = StableDiffusionPipeline.from_pretrained(model_path)
        pipe.to("cuda" if torch.cuda.is_available() else "cpu")
        image = pipe(prompt).images[0]
        return image
    except Exception as e:
        logging.error(f"Error generating image: {str(e)}")
        return None

def save_image(image, path):
    try:
        os.makedirs(os.path.dirname(path), exist_ok=True)
        image.save(path)
        logging.info(f"Image saved: {path}")
    except Exception as e:
        logging.error(f"Error saving image: {str(e)}")

def process_problem(problem, tokenizer, bert_model, model_path="CompVis/stable-diffusion-v1-4"):
    try:
        problem_id = problem["Problem_ID"]
        problem_statement = problem["Problem_Statement"]
        concepts = problem["Concepts_Covered"]
        difficulty = classify_problem(problem_statement, concepts, tokenizer, bert_model)
        image_prompt = problem["3D_Diagram_Description"]
        
        logging.info(f"Generating image for Problem {problem_id} ({difficulty} difficulty)...")
        generated_image = generate_image(image_prompt, model_path)
        if generated_image:
            save_image(generated_image, f"generated_images/problem_{problem_id}.png")
    except Exception as e:
        logging.error(f"Error processing problem {problem_id}: {str(e)}")

# Load models
tokenizer, bert_model = load_bert_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Generate images in parallel
with ThreadPoolExecutor(max_workers=4) as executor:
    for problem in dataset:
        executor.submit(process_problem, problem, tokenizer, bert_model)

logging.info("Batch image generation completed.")


In [ ]:
import os
import pandas as pd
import torch
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model for evaluation
def load_clip_model():
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    return model, processor

def evaluate_image(image_path, prompt, model, processor):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=[prompt], images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    score = outputs.logits_per_image.item()
    return score

# Load generated images and dataset
dataset_path = "updated_physics_problems_dataset_2.json"
image_folder = "generated_images"

torch_device = "cuda" if torch.cuda.is_available() else "cpu"
model, processor = load_clip_model()
model.to(torch_device)

with open(dataset_path, "r", encoding="utf-8") as f:
    dataset = pd.read_json(f)

evaluation_results = []
for problem in dataset.itertuples():
    problem_id = problem.Problem_ID
    image_path = os.path.join(image_folder, f"problem_{problem_id}.png")
    prompt = problem._asdict().get("3D_Diagram_Description", "")
    
    if os.path.exists(image_path):
        score = evaluate_image(image_path, prompt, model, processor)
        evaluation_results.append({"Problem_ID": problem_id, "Image_Path": image_path, "Score": score})

# Save results
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("image_evaluation_results.csv", index=False)
print("Evaluation complete. Results saved to image_evaluation_results.csv")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load evaluation results
df = pd.read_csv("image_evaluation_results.csv")

# Plot distribution of CLIP similarity scores
def plot_evaluation_scores(df):
    plt.figure(figsize=(10, 5))
    sns.histplot(df["Score"], bins=20, kde=True, color='blue')
    plt.xlabel("CLIP Similarity Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of Image Evaluation Scores")
    plt.show()

# Scatter plot of Problem IDs vs Scores
def plot_problem_scores(df):
    plt.figure(figsize=(12, 6))
    sns.scatterplot(x=df["Problem_ID"], y=df["Score"], hue=df["Score"], palette="viridis", size=df["Score"], sizes=(20, 200))
    plt.xlabel("Problem ID")
    plt.ylabel("CLIP Similarity Score")
    plt.title("Evaluation Scores Across Problems")
    plt.legend(title="Score", bbox_to_anchor=(1, 1))
    plt.show()

# Run plots
plot_evaluation_scores(df)
plot_problem_scores(df)


In [ ]:
import pandas as pd

# Load evaluation results
df = pd.read_csv("image_evaluation_results.csv")

# Generate summary statistics
summary = df["Score"].describe()

# Identify best and worst performing images
best_images = df.nlargest(5, "Score")
worst_images = df.nsmallest(5, "Score")

# Save report to a text file
report_path = "image_evaluation_report.txt"
with open(report_path, "w") as f:
    f.write("Image Evaluation Report\n")
    f.write("========================\n\n")
    f.write("Summary Statistics:\n")
    f.write(str(summary) + "\n\n")
    f.write("Top 5 Best Performing Images:\n")
    f.write(best_images.to_string(index=False) + "\n\n")
    f.write("Top 5 Worst Performing Images:\n")
    f.write(worst_images.to_string(index=False) + "\n")

print(f"Report generated and saved to {report_path}")


In [ ]:
import torch
import os
from torchvision import transforms
from PIL import Image
from diffusers import StableDiffusionPipeline
from torchvision.utils import save_image

# Load CycleGAN model for refinement
def load_cyclegan_model(model_path="cyclegan_refinement.pth"):
    model = torch.jit.load(model_path)
    model.eval()
    return model

# Load ControlNet for structured enhancement
def load_controlnet_model(model_path="controlnet.pth"):
    model = torch.jit.load(model_path)
    model.eval()
    return model

# Apply CycleGAN for refinement
def apply_cyclegan(image, model):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    input_image = transform(image).unsqueeze(0)
    with torch.no_grad():
        refined_image = model(input_image).squeeze().detach().cpu()
    refined_image = transforms.ToPILImage()(refined_image)
    return refined_image

# Apply ControlNet for structured enhancement
def apply_controlnet(image, model):
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    input_image = transform(image).unsqueeze(0)
    with torch.no_grad():
        enhanced_image = model(input_image).squeeze().detach().cpu()
    enhanced_image = transforms.ToPILImage()(enhanced_image)
    return enhanced_image

# Load models
cyclegan_model = load_cyclegan_model()
controlnet_model = load_controlnet_model()

# Process images
image_folder = "generated_images"
refined_folder = "refined_images"
os.makedirs(refined_folder, exist_ok=True)

for filename in os.listdir(image_folder):
    image_path = os.path.join(image_folder, filename)
    image = Image.open(image_path).convert("RGB")
    
    print(f"Refining {filename} with CycleGAN...")
    refined_image = apply_cyclegan(image, cyclegan_model)
    
    print(f"Enhancing {filename} with ControlNet...")
    final_image = apply_controlnet(refined_image, controlnet_model)
    
    final_image.save(os.path.join(refined_folder, filename))
    print(f"Saved refined image: {os.path.join(refined_folder, filename)}")


In [ ]:
import os
import matplotlib.pyplot as plt
from PIL import Image

def load_images(image_folder, refined_folder, num_images=5):
    image_pairs = []
    filenames = sorted(os.listdir(image_folder))[:num_images]
    for filename in filenames:
        orig_path = os.path.join(image_folder, filename)
        refined_path = os.path.join(refined_folder, filename)
        if os.path.exists(refined_path):
            orig_image = Image.open(orig_path).convert("RGB")
            refined_image = Image.open(refined_path).convert("RGB")
            image_pairs.append((filename, orig_image, refined_image))
    return image_pairs

def plot_comparison(image_pairs):
    fig, axes = plt.subplots(len(image_pairs), 2, figsize=(10, 5 * len(image_pairs)))
    fig.suptitle("Original vs. Refined Images", fontsize=16)
    
    for ax_row, (filename, orig_img, refined_img) in zip(axes, image_pairs):
        ax_row[0].imshow(orig_img)
        ax_row[0].set_title(f"Original - {filename}")
        ax_row[0].axis("off")
        
        ax_row[1].imshow(refined_img)
        ax_row[1].set_title(f"Refined - {filename}")
        ax_row[1].axis("off")
    
    plt.tight_layout()
    plt.show()

# Load and compare images
image_folder = "generated_images"
refined_folder = "refined_images"
image_pairs = load_images(image_folder, refined_folder, num_images=5)
plot_comparison(image_pairs)


In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model for evaluation
def load_clip_model():
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    return model, processor

def compute_clip_similarity(image_path, prompt, model, processor):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(text=[prompt], images=image, return_tensors="pt", padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits_per_image.item()

# Load models
torch_device = "cuda" if torch.cuda.is_available() else "cpu"
model, processor = load_clip_model()
model.to(torch_device)

# Load dataset
dataset_path = "updated_physics_problems_dataset_2.json"
image_folder = "generated_images"
refined_folder = "refined_images"

df = pd.read_json(dataset_path)

# Evaluate images
evaluation_results = []
for problem in df.itertuples():
    problem_id = problem.Problem_ID
    prompt = problem._asdict().get("3D_Diagram_Description", "")
    orig_image_path = os.path.join(image_folder, f"problem_{problem_id}.png")
    refined_image_path = os.path.join(refined_folder, f"problem_{problem_id}.png")
    
    if os.path.exists(orig_image_path) and os.path.exists(refined_image_path):
        orig_score = compute_clip_similarity(orig_image_path, prompt, model, processor)
        refined_score = compute_clip_similarity(refined_image_path, prompt, model, processor)
        improvement = refined_score - orig_score
        evaluation_results.append({
            "Problem_ID": problem_id,
            "Original_Score": orig_score,
            "Refined_Score": refined_score,
            "Improvement": improvement
        })

# Save results
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("refinement_quality_scores.csv", index=False)
print("Refinement quality evaluation completed. Results saved to refinement_quality_scores.csv")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load refinement quality scores
df = pd.read_csv("refinement_quality_scores.csv")

# Plot distribution of improvement scores
def plot_improvement_distribution(df):
    plt.figure(figsize=(10, 5))
    sns.histplot(df["Improvement"], bins=20, kde=True, color='blue')
    plt.xlabel("Improvement in CLIP Similarity Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of Refinement Improvements")
    plt.show()

# Scatter plot of Problem IDs vs Improvement
def plot_improvement_scatter(df):
    plt.figure(figsize=(12, 6))
    sns.scatterplot(x=df["Problem_ID"], y=df["Improvement"], hue=df["Improvement"], palette="coolwarm", size=df["Improvement"], sizes=(20, 200))
    plt.xlabel("Problem ID")
    plt.ylabel("Improvement in CLIP Score")
    plt.title("Improvement in Refinement Across Problems")
    plt.legend(title="Improvement Score", bbox_to_anchor=(1, 1))
    plt.show()

# Run plots
plot_improvement_distribution(df)
plot_improvement_scatter(df)


In [ ]:
import sympy as sp
import json
import matplotlib.pyplot as plt
import numpy as np

# Load generated solutions
df = json.load(open("flan_t5_generated_solutions.json", "r"))

def check_algebraic_correctness(expression, expected_result):
    try:
        expr = sp.sympify(expression)
        expected = sp.sympify(expected_result)
        return sp.simplify(expr - expected) == 0
    except Exception as e:
        print(f"Error checking correctness: {e}")
        return False

def generate_problem_graph(problem_statement, solution_expression):
    x = sp.symbols('x')
    try:
        expr = sp.sympify(solution_expression)
        f_lambdified = sp.lambdify(x, expr, 'numpy')
        
        x_vals = np.linspace(-10, 10, 400)
        y_vals = f_lambdified(x_vals)
        
        plt.figure(figsize=(8,5))
        plt.plot(x_vals, y_vals, label=f"y = {solution_expression}", color='blue')
        plt.axhline(0, color='black', linewidth=0.5)
        plt.axvline(0, color='black', linewidth=0.5)
        plt.grid(True, linestyle='--', linewidth=0.5)
        plt.title(f"Graph for: {problem_statement}")
        plt.legend()
        plt.show()
    except Exception as e:
        print(f"Error generating graph: {e}")

# Iterate over generated solutions to check algebraic correctness
for problem in df:
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    generated_solution = problem["Generated_Solution"]
    expected_answer = problem.get("Final_Answer", "")
    
    if check_algebraic_correctness(generated_solution, expected_answer):
        print(f"Problem {problem_id}: Correct Algebraic Solution")
    else:
        print(f"Problem {problem_id}: Incorrect Algebraic Solution")
    
    generate_problem_graph(problem_statement, generated_solution)


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
from skimage.metrics import structural_similarity as ssim

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

def setup_pybullet():
    p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.8)

def generate_simulation(problem_statement):
    setup_pybullet()
    plane_id = p.loadURDF("plane.urdf")
    box_id = p.loadURDF("r2d2.urdf", [0, 0, 1])
    
    video_frames = []
    for i in range(100):
        p.stepSimulation()
        frame = p.getCameraImage(320, 240)[2]
        frame = np.array(frame, dtype=np.uint8)
        video_frames.append(frame)
    
    p.disconnect()
    return video_frames

def save_simulation_video(frames, output_path="simulation_output.avi"):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))
    
    for frame in frames:
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    out.release()
    print(f"Simulation video saved at {output_path}")

def calculate_ssim(original_image, generated_image):
    original_gray = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    generated_gray = cv2.cvtColor(generated_image, cv2.COLOR_BGR2GRAY)
    return ssim(original_gray, generated_gray)

# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    
    print(f"Generating simulation for Problem {problem_id}...")
    frames = generate_simulation(problem_statement)
    save_simulation_video(frames, f"simulation_videos/problem_{problem_id}.avi")
    
    # Validate last frame against first frame using SSIM
    similarity_score = calculate_ssim(frames[0], frames[-1])
    print(f"Problem {problem_id} - SSIM Score: {similarity_score:.4f}")


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, tokenizer, model):
    inputs = tokenizer(problem_statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_explanation(problem_statement, tokenizer, model, max_length=200):
    input_text = f"Explain: {problem_statement}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def setup_pybullet():
    p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.8)

def generate_simulation(simulation_description):
    setup_pybullet()
    plane_id = p.loadURDF("plane.urdf")
    box_id = p.loadURDF("r2d2.urdf", [0, 0, 1])
    
    video_frames = []
    for _ in range(100):
        p.stepSimulation()
        frame = p.getCameraImage(320, 240)[2]
        frame = np.array(frame, dtype=np.uint8)
        video_frames.append(frame)
    
    p.disconnect()
    return video_frames

def save_simulation_video(frames, output_path):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))
    
    for frame in frames:
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    out.release()
    print(f"Simulation video saved at {output_path}")

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    simulation_description = problem.get("Simulation_Description", "")
    
    difficulty = classify_problem(problem_statement, tokenizer_bert, model_bert)
    explanation = generate_explanation(problem_statement, tokenizer_flan, model_flan)
    print(f"Problem {problem_id} - Difficulty: {difficulty}\nExplanation: {explanation}")
    
    print(f"Generating simulation for Problem {problem_id}...")
    frames = generate_simulation(simulation_description)
    save_simulation_video(frames, f"simulation_videos/problem_{problem_id}.avi")


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
from skimage.metrics import structural_similarity as ssim
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, tokenizer, model):
    inputs = tokenizer(problem_statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_explanation(problem_statement, tokenizer, model, max_length=200):
    input_text = f"Explain: {problem_statement}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def setup_pybullet():
    p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.8)

def generate_simulation(simulation_description):
    setup_pybullet()
    plane_id = p.loadURDF("plane.urdf")
    box_id = p.loadURDF("r2d2.urdf", [0, 0, 1])
    
    video_frames = []
    for _ in range(100):
        p.stepSimulation()
        frame = p.getCameraImage(320, 240)[2]
        frame = np.array(frame, dtype=np.uint8)
        video_frames.append(frame)
    
    p.disconnect()
    return video_frames

def save_simulation_video(frames, output_path):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))
    
    for frame in frames:
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    out.release()
    print(f"Simulation video saved at {output_path}")

def calculate_ssim(original_frame, generated_frame):
    original_gray = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)
    generated_gray = cv2.cvtColor(generated_frame, cv2.COLOR_BGR2GRAY)
    return ssim(original_gray, generated_gray)

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    simulation_description = problem.get("Simulation_Description", "")
    
    difficulty = classify_problem(problem_statement, tokenizer_bert, model_bert)
    explanation = generate_explanation(problem_statement, tokenizer_flan, model_flan)
    print(f"Problem {problem_id} - Difficulty: {difficulty}\nExplanation: {explanation}")
    
    print(f"Generating simulation for Problem {problem_id}...")
    frames = generate_simulation(simulation_description)
    save_simulation_video(frames, f"simulation_videos/problem_{problem_id}.avi")
    
    # Validate simulation using SSIM
    similarity_score = calculate_ssim(frames[0], frames[-1])
    print(f"Problem {problem_id} - SSIM Score: {similarity_score:.4f}")

In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, tokenizer, model):
    inputs = tokenizer(problem_statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_explanation(problem_statement, tokenizer, model, max_length=200):
    input_text = f"Explain: {problem_statement}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def setup_pybullet():
    p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.8)

def generate_simulation(simulation_description):
    setup_pybullet()
    plane_id = p.loadURDF("plane.urdf")
    box_id = p.loadURDF("r2d2.urdf", [0, 0, 1])
    
    video_frames = []
    for _ in range(100):
        p.stepSimulation()
        frame = p.getCameraImage(320, 240)[2]
        frame = np.array(frame, dtype=np.uint8)
        video_frames.append(frame)
    
    p.disconnect()
    return video_frames

def save_simulation_video(frames, output_path):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))
    
    for frame in frames:
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    out.release()
    print(f"Simulation video saved at {output_path}")

def calculate_ssim(original_frame, generated_frame):
    original_gray = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)
    generated_gray = cv2.cvtColor(generated_frame, cv2.COLOR_BGR2GRAY)
    return ssim(original_gray, generated_gray)

def plot_ssim_scores(ssim_scores):
    plt.figure(figsize=(10, 5))
    sns.histplot(ssim_scores, bins=10, kde=True, color='blue')
    plt.xlabel("SSIM Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of SSIM Scores Across Simulations")
    plt.show()

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

ssim_scores = []
# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    simulation_description = problem.get("Simulation_Description", "")
    
    difficulty = classify_problem(problem_statement, tokenizer_bert, model_bert)
    explanation = generate_explanation(problem_statement, tokenizer_flan, model_flan)
    print(f"Problem {problem_id} - Difficulty: {difficulty}\nExplanation: {explanation}")
    
    print(f"Generating simulation for Problem {problem_id}...")
    frames = generate_simulation(simulation_description)
    save_simulation_video(frames, f"simulation_videos/problem_{problem_id}.avi")
    
    # Validate simulation using SSIM
    similarity_score = calculate_ssim(frames[0], frames[-1])
    ssim_scores.append(similarity_score)
    print(f"Problem {problem_id} - SSIM Score: {similarity_score:.4f}")

# Plot SSIM scores
total_problems = len(ssim_scores)
if total_problems > 0:
    plot_ssim_scores(ssim_scores)


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def classify_problem(problem_statement, tokenizer, model):
    inputs = tokenizer(problem_statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_explanation(problem_statement, tokenizer, model, max_length=200):
    input_text = f"Explain: {problem_statement}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def setup_pybullet():
    p.connect(p.DIRECT)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.8)

def generate_simulation(simulation_description):
    setup_pybullet()
    plane_id = p.loadURDF("plane.urdf")
    box_id = p.loadURDF("r2d2.urdf", [0, 0, 1])
    
    video_frames = []
    for _ in range(100):
        p.stepSimulation()
        frame = p.getCameraImage(320, 240)[2]
        frame = np.array(frame, dtype=np.uint8)
        video_frames.append(frame)
    
    p.disconnect()
    return video_frames

def save_simulation_video(frames, output_path):
    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*"XVID")
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (width, height))
    
    for frame in frames:
        out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
    
    out.release()
    print(f"Simulation video saved at {output_path}")

def calculate_ssim(original_frame, generated_frame):
    original_gray = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)
    generated_gray = cv2.cvtColor(generated_frame, cv2.COLOR_BGR2GRAY)
    return ssim(original_gray, generated_gray)

def plot_ssim_scores(ssim_scores):
    plt.figure(figsize=(10, 5))
    sns.histplot(ssim_scores, bins=10, kde=True, color='blue')
    plt.xlabel("SSIM Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of SSIM Scores Across Simulations")
    plt.show()

def generate_ssim_report(ssim_scores, output_file="ssim_report.txt"):
    with open(output_file, "w") as f:
        f.write("SSIM Score Report\n")
        f.write("===================\n\n")
        f.write(f"Total Simulations Evaluated: {len(ssim_scores)}\n")
        f.write(f"Average SSIM Score: {np.mean(ssim_scores):.4f}\n")
        f.write(f"Max SSIM Score: {np.max(ssim_scores):.4f}\n")
        f.write(f"Min SSIM Score: {np.min(ssim_scores):.4f}\n")
    print(f"SSIM report saved at {output_file}")

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

ssim_scores = []
# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    simulation_description = problem.get("Simulation_Description", "")
    
    difficulty = classify_problem(problem_statement, tokenizer_bert, model_bert)
    explanation = generate_explanation(problem_statement, tokenizer_flan, model_flan)
    print(f"Problem {problem_id} - Difficulty: {difficulty}\nExplanation: {explanation}")
    
    print(f"Generating simulation for Problem {problem_id}...")
    frames = generate_simulation(simulation_description)
    save_simulation_video(frames, f"simulation_videos/problem_{problem_id}.avi")
    
    # Validate simulation using SSIM
    similarity_score = calculate_ssim(frames[0], frames[-1])
    ssim_scores.append(similarity_score)
    print(f"Problem {problem_id} - SSIM Score: {similarity_score:.4f}")

# Plot SSIM scores
if ssim_scores:
    plot_ssim_scores(ssim_scores)
    generate_ssim_report(ssim_scores)


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
from manim import *
import bpy  # Blender Python API
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def setup_blender_simulation():
    bpy.ops.wm.read_factory_settings(use_empty=True)
    bpy.ops.mesh.primitive_cube_add(size=2, location=(0, 0, 1))
    bpy.ops.rigidbody.object_add()
    bpy.ops.mesh.primitive_plane_add(size=10, location=(0, 0, 0))
    bpy.ops.rigidbody.object_add()
    bpy.context.object.rigid_body.type = 'PASSIVE'

def run_blender_simulation(output_path="blender_simulation.mp4"):
    bpy.context.scene.frame_start = 1
    bpy.context.scene.frame_end = 100
    bpy.context.scene.render.filepath = output_path
    bpy.ops.render.render(animation=True)
    print(f"Blender physics simulation saved at {output_path}")

def classify_problem(problem_statement, tokenizer, model):
    inputs = tokenizer(problem_statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_explanation(problem_statement, tokenizer, model, max_length=200):
    input_text = f"Explain: {problem_statement}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def calculate_ssim(original_frame, generated_frame):
    original_gray = cv2.cvtColor(original_frame, cv2.COLOR_BGR2GRAY)
    generated_gray = cv2.cvtColor(generated_frame, cv2.COLOR_BGR2GRAY)
    return ssim(original_gray, generated_gray)

def plot_ssim_scores(ssim_scores):
    plt.figure(figsize=(10, 5))
    sns.histplot(ssim_scores, bins=10, kde=True, color='blue')
    plt.xlabel("SSIM Score")
    plt.ylabel("Frequency")
    plt.title("Distribution of SSIM Scores Across Simulations")
    plt.show()

def generate_ssim_report(ssim_scores, output_file="ssim_report.txt"):
    with open(output_file, "w") as f:
        f.write("SSIM Score Report\n")
        f.write("===================\n\n")
        f.write(f"Total Simulations Evaluated: {len(ssim_scores)}\n")
        f.write(f"Average SSIM Score: {np.mean(ssim_scores):.4f}\n")
        f.write(f"Max SSIM Score: {np.max(ssim_scores):.4f}\n")
        f.write(f"Min SSIM Score: {np.min(ssim_scores):.4f}\n")
    print(f"SSIM report saved at {output_file}")

class MathVisualization(Scene):
    def construct(self):
        equation = MathTex("F = ma")
        self.play(Write(equation))
        self.wait(2)

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

ssim_scores = []
# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    
    difficulty = classify_problem(problem_statement, tokenizer_bert, model_bert)
    explanation = generate_explanation(problem_statement, tokenizer_flan, model_flan)
    print(f"Problem {problem_id} - Difficulty: {difficulty}\nExplanation: {explanation}")
    
    print(f"Running Blender physics simulation for Problem {problem_id}...")
    setup_blender_simulation()
    run_blender_simulation(f"blender_simulations/problem_{problem_id}.mp4")


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
from manim import *
import bpy  # Blender Python API
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def setup_blender_simulation():
    bpy.ops.wm.read_factory_settings(use_empty=True)
    bpy.ops.mesh.primitive_cube_add(size=2, location=(0, 0, 1))
    bpy.ops.rigidbody.object_add()
    bpy.ops.mesh.primitive_plane_add(size=10, location=(0, 0, 0))
    bpy.ops.rigidbody.object_add()
    bpy.context.object.rigid_body.type = 'PASSIVE'

def run_blender_simulation(output_path="blender_simulation.mp4"):
    bpy.context.scene.frame_start = 1
    bpy.context.scene.frame_end = 100
    bpy.context.scene.render.filepath = output_path
    bpy.ops.render.render(animation=True)
    print(f"Blender physics simulation saved at {output_path}")

def classify_problem(problem_statement, tokenizer, model):
    inputs = tokenizer(problem_statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_explanation(problem_statement, tokenizer, model, max_length=200):
    input_text = f"Explain: {problem_statement}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def compare_pybullet_blender(pybullet_video, blender_video):
    print(f"Comparing PyBullet Simulation: {pybullet_video} with Blender Simulation: {blender_video}")
    pyb_vid = cv2.VideoCapture(pybullet_video)
    blend_vid = cv2.VideoCapture(blender_video)
    ssim_scores = []
    
    while True:
        ret1, frame1 = pyb_vid.read()
        ret2, frame2 = blend_vid.read()
        
        if not ret1 or not ret2:
            break
        
        frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        score = ssim(frame1_gray, frame2_gray)
        ssim_scores.append(score)
    
    pyb_vid.release()
    blend_vid.release()
    
    avg_ssim = np.mean(ssim_scores)
    print(f"Average SSIM Score between PyBullet and Blender: {avg_ssim:.4f}")
    return avg_ssim

def generate_ssim_report(ssim_scores, output_file="ssim_report.txt"):
    with open(output_file, "w") as f:
        f.write("SSIM Score Report\n")
        f.write("===================\n\n")
        f.write(f"Total Simulations Evaluated: {len(ssim_scores)}\n")
        f.write(f"Average SSIM Score: {np.mean(ssim_scores):.4f}\n")
        f.write(f"Max SSIM Score: {np.max(ssim_scores):.4f}\n")
        f.write(f"Min SSIM Score: {np.min(ssim_scores):.4f}\n")
    print(f"SSIM report saved at {output_file}")

class MathVisualization(Scene):
    def construct(self):
        equation = MathTex("F = ma")
        self.play(Write(equation))
        self.wait(2)

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

ssim_scores = []
# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    
    difficulty = classify_problem(problem_statement, tokenizer_bert, model_bert)
    explanation = generate_explanation(problem_statement, tokenizer_flan, model_flan)
    print(f"Problem {problem_id} - Difficulty: {difficulty}\nExplanation: {explanation}")
    
    print(f"Running Blender physics simulation for Problem {problem_id}...")
    setup_blender_simulation()
    blender_video = f"blender_simulations/problem_{problem_id}.mp4"
    run_blender_simulation(blender_video)
    
    pybullet_video = f"pybullet_simulations/problem_{problem_id}.mp4"  # Placeholder for future PyBullet video
    avg_ssim = compare_pybullet_blender(pybullet_video, blender_video)
    ssim_scores.append(avg_ssim)
    
if ssim_scores:
    generate_ssim_report(ssim_scores)


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
from manim import *
import bpy  # Blender Python API
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def setup_blender_simulation():
    bpy.ops.wm.read_factory_settings(use_empty=True)
    bpy.ops.mesh.primitive_cube_add(size=2, location=(0, 0, 1))
    bpy.ops.rigidbody.object_add()
    bpy.ops.mesh.primitive_plane_add(size=10, location=(0, 0, 0))
    bpy.ops.rigidbody.object_add()
    bpy.context.object.rigid_body.type = 'PASSIVE'

def run_blender_simulation(output_path="blender_simulation.mp4"):
    bpy.context.scene.frame_start = 1
    bpy.context.scene.frame_end = 100
    bpy.context.scene.render.filepath = output_path
    bpy.ops.render.render(animation=True)
    print(f"Blender physics simulation saved at {output_path}")

def classify_problem(problem_statement, tokenizer, model):
    inputs = tokenizer(problem_statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    label_map = {0: "Easy", 1: "Medium", 2: "Hard"}
    return label_map[prediction]

def generate_explanation(problem_statement, tokenizer, model, max_length=200):
    input_text = f"Explain: {problem_statement}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        output = model.generate(**inputs, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def compare_pybullet_blender(pybullet_video, blender_video):
    print(f"Comparing PyBullet Simulation: {pybullet_video} with Blender Simulation: {blender_video}")
    pyb_vid = cv2.VideoCapture(pybullet_video)
    blend_vid = cv2.VideoCapture(blender_video)
    ssim_scores = []
    
    while True:
        ret1, frame1 = pyb_vid.read()
        ret2, frame2 = blend_vid.read()
        
        if not ret1 or not ret2:
            break
        
        frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        score = ssim(frame1_gray, frame2_gray)
        ssim_scores.append(score)
    
    pyb_vid.release()
    blend_vid.release()
    
    avg_ssim = np.mean(ssim_scores)
    print(f"Average SSIM Score between PyBullet and Blender: {avg_ssim:.4f}")
    return avg_ssim

def plot_ssim_comparison(ssim_scores):
    plt.figure(figsize=(10, 5))
    sns.histplot(ssim_scores, bins=10, kde=True, color='blue')
    plt.xlabel("SSIM Score")
    plt.ylabel("Frequency")
    plt.title("SSIM Comparison: PyBullet vs Blender Simulations")
    plt.show()

def generate_ssim_report(ssim_scores, output_file="ssim_report.txt"):
    with open(output_file, "w") as f:
        f.write("SSIM Score Report\n")
        f.write("===================\n\n")
        f.write(f"Total Simulations Evaluated: {len(ssim_scores)}\n")
        f.write(f"Average SSIM Score: {np.mean(ssim_scores):.4f}\n")
        f.write(f"Max SSIM Score: {np.max(ssim_scores):.4f}\n")
        f.write(f"Min SSIM Score: {np.min(ssim_scores):.4f}\n")
    print(f"SSIM report saved at {output_file}")

class MathVisualization(Scene):
    def construct(self):
        equation = MathTex("F = ma")
        self.play(Write(equation))
        self.wait(2)

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

ssim_scores = []
# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    problem_statement = problem["Problem_Statement"]
    
    difficulty = classify_problem(problem_statement, tokenizer_bert, model_bert)
    explanation = generate_explanation(problem_statement, tokenizer_flan, model_flan)
    print(f"Problem {problem_id} - Difficulty: {difficulty}\nExplanation: {explanation}")
    
    print(f"Running Blender physics simulation for Problem {problem_id}...")
    setup_blender_simulation()
    blender_video = f"blender_simulations/problem_{problem_id}.mp4"
    run_blender_simulation(blender_video)
    
    pybullet_video = f"pybullet_simulations/problem_{problem_id}.mp4"  # Placeholder for future PyBullet video
    avg_ssim = compare_pybullet_blender(pybullet_video, blender_video)
    ssim_scores.append(avg_ssim)
    
if ssim_scores:
    plot_ssim_comparison(ssim_scores)
    generate_ssim_report(ssim_scores)


In [ ]:
import pybullet as p
import pybullet_data
import cv2
import numpy as np
import json
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import structural_similarity as ssim
from manim import *
import bpy  # Blender Python API
from transformers import BertTokenizer, BertForSequenceClassification, T5Tokenizer, T5ForConditionalGeneration, LlamaTokenizer, LlamaForCausalLM

# Load models
def load_bert_model(model_path="bert-base-uncased"):
    tokenizer = BertTokenizer.from_pretrained(model_path)
    model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
    return tokenizer, model

def load_flan_t5_model(model_path="google/flan-t5-large"):
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = T5ForConditionalGeneration.from_pretrained(model_path)
    return tokenizer, model

def load_llama_model(model_path="meta-llama/Llama-2-7b-chat-hf"):
    tokenizer = LlamaTokenizer.from_pretrained(model_path)
    model = LlamaForCausalLM.from_pretrained(model_path)
    return tokenizer, model

def compare_pybullet_blender(pybullet_video, blender_video):
    print(f"Comparing PyBullet Simulation: {pybullet_video} with Blender Simulation: {blender_video}")
    pyb_vid = cv2.VideoCapture(pybullet_video)
    blend_vid = cv2.VideoCapture(blender_video)
    ssim_scores = []
    
    while True:
        ret1, frame1 = pyb_vid.read()
        ret2, frame2 = blend_vid.read()
        
        if not ret1 or not ret2:
            break
        
        frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        
        score = ssim(frame1_gray, frame2_gray)
        ssim_scores.append(score)
    
    pyb_vid.release()
    blend_vid.release()
    
    avg_ssim = np.mean(ssim_scores)
    print(f"Average SSIM Score between PyBullet and Blender: {avg_ssim:.4f}")
    return avg_ssim

def rank_simulations(simulation_scores):
    ranked_simulations = sorted(simulation_scores, key=lambda x: x["ssim"], reverse=True)
    print("\nRanked Simulations (Best to Worst):")
    for rank, sim in enumerate(ranked_simulations, start=1):
        print(f"Rank {rank}: Problem {sim['problem_id']} - SSIM: {sim['ssim']:.4f}")

def generate_ssim_report(ssim_scores, output_file="ssim_report.txt"):
    with open(output_file, "w") as f:
        f.write("SSIM Score Report\n")
        f.write("===================\n\n")
        f.write(f"Total Simulations Evaluated: {len(ssim_scores)}\n")
        f.write(f"Average SSIM Score: {np.mean(ssim_scores):.4f}\n")
        f.write(f"Max SSIM Score: {np.max(ssim_scores):.4f}\n")
        f.write(f"Min SSIM Score: {np.min(ssim_scores):.4f}\n")
    print(f"SSIM report saved at {output_file}")

# Load models
tokenizer_bert, model_bert = load_bert_model()
tokenizer_flan, model_flan = load_flan_t5_model()
tokenizer_llama, model_llama = load_llama_model()

# Load dataset
with open("updated_physics_problems_dataset_2.json", "r", encoding="utf-8") as f:
    dataset = json.load(f)

ssim_scores = []
simulation_results = []
# Process dataset
for problem in dataset[:5]:  # Limiting to first 5 for efficiency
    problem_id = problem["Problem_ID"]
    
    print(f"Comparing PyBullet and Blender physics simulations for Problem {problem_id}...")
    pybullet_video = f"pybullet_simulations/problem_{problem_id}.mp4"
    blender_video = f"blender_simulations/problem_{problem_id}.mp4"
    
    avg_ssim = compare_pybullet_blender(pybullet_video, blender_video)
    ssim_scores.append(avg_ssim)
    simulation_results.append({"problem_id": problem_id, "ssim": avg_ssim})
    
if ssim_scores:
    generate_ssim_report(ssim_scores)
    rank_simulations(simulation_results)
